# ErLang Notepad
* [About Notebook](#section1) - Running Erlang tests from a stanbdard IPython kernel
* [Erlang Data Structures](#section2) 
    * [Tuples](#section2)
    * [Lists](#section2b)
    * [Quicksort](#quicksort)
    * [Maps](#maps)
* [Timeit Module](#timeit)
* [Functions - Pattern Matching](#section3)
    * [Functional Closures](#funcClosure)
    * [Partial function application in Erlang](#partialApplication)
* [Concurrent Programming](#concurrent)

In [2]:
%autosave 30
%cd ~/erlang/tests

Autosaving every 30 seconds
/Users/mkersh/erlang/tests


<a id="section1"></a>
## About Notebook
This is my notebook for experimenting with Erlang.

I was trying to use the official [ierlang iPython kernel](https://github.com/robbielynch/ierlang) but am not able to get this running on my MacBook Pro.

So instead I am using magic cell commands in a standard IPython notebook. These allow you to run code and commands other than just python code in cells. Here's a list of the magic cell commands that I am using:

* %autosave 30 - autosave the notebook every 30 seconds
* %cd &lt;testDir&gt; - Change the current directory
    * NOTE: Above two are not strictly speaking magic cell commands (these begin with %%)
        * Difference is that a magic cell command will use the content of the cell as input for the command 
* %load &lt;filename&gt; - Loads the content of the file into the cell
    * No need for this but I thought I'd mention it just in case I do need it.
* %%writefile &lt;filename&gt;
    * This is our first true magic cell command and it really is magic!
    * When executed (using shift+RETURN) it will write the value of the cell (after the %%writefile line) as the contents of the file.
    * This therefore gives us the basic mechanism for creating Erlang files from the cells of the Notebook
* %%script erl - Start the Erlang Run Time System
    * From within this cell we can then apply Erlang command:
        * c(&lt;module&gt;).  - To compile a file
        * &lt;module&gt;:main(). - to run the contents
    * Output appears as output of the cell
    
The above are all we need to be able to run and test Erlang concepts from a standard IPython Jupyter notebook.

The disadvantages of this approach are:
* There is no history across cells
    * Not sure how the ierlang kernel would have improved this though because Erlang works off files
* I need 2 cells for each test
    
### First Examples of how to use this approach 

The below cells show my approach to running Erlang from a standard IPython kernel


In [3]:
%%writefile first22.erl
-module(first22).
-export([double/1,main/0,main/1,start/0]).

double(X) ->
    times(X, 2).

times(X, N) ->
    X * N.

main(_) ->
    main().

main() ->
    A = [1,2,3,4],
    io:format("Change the file here Hello world ~p!~n", [A]),
    io:format("Double a number ~p~n", [double(10)]).

start() ->
    main().

 

Overwriting first22.erl


In [4]:
%%script erl
c(first22).
first22:main().


Eshell V10.2.3  (abort with ^G)
1> {ok,first22}
2> Change the file here Hello world [1,2,3,4]!
Double a number 20
ok
3> *** Terminating erlang (nonode@nohost)


#### Using the same file for each cell

The next 2 examples show the use of a single temporary file (which will be overwritten) for each cell

In [5]:
%%writefile ertest.erl
-module(ertest).
-export([main/0]).


main() ->
    A = [1,2,3,4],
    io:format("test 1 ~p!~n", [A]).



Overwriting ertest.erl


With this approach the Erlang Run Time cell can just be copied for each test

In [6]:
%%script erl
c(ertest).
ertest:main().

Eshell V10.2.3  (abort with ^G)
1> {ok,ertest}
2> test 1 [1,2,3,4]!
ok
3> *** Terminating erlang (nonode@nohost)


In [7]:
%%writefile ertest.erl
-module(ertest).
-export([main/0]).

main() ->
    A = [1,2,3,4, ttt, yyy, uuu],
    io:format("test 2222 ~p!~n", [A]).


Overwriting ertest.erl


In [8]:
%%script erl
c(ertest).
ertest:main().

Eshell V10.2.3  (abort with ^G)
1> {ok,ertest}
2> test 2222 [1,2,3,4,ttt,yyy,uuu]!
ok
3> *** Terminating erlang (nonode@nohost)


<a id="section2"></a>
## Erlang Data Structures

### Tuples

In [8]:
%%writefile ertest.erl
-module(ertest).
-export([main/0]).

% This first Test shows the tuple data structure in Erlang
% It also shows that tuples can have mixed elements in them
% This ifdef is a kludge to get around the fact that you can only have
% single line comments in Erlang

main() ->
    A = {1,2,3,4,one,two,three,four, "mark", <<"Kershaw">>, 56.67},
    io:format("Tuple with mixed content~p~n", [A]),
    io:format("element from tuple: ~p~n", [element(5,A)]).


Overwriting ertest.erl


In [9]:
%%script erl
c(ertest).
ertest:main().

Eshell V10.2.3  (abort with ^G)
1> {ok,ertest}
2> Tuple with mixed content{1,2,3,4,one,two,three,four,"mark",<<"Kershaw">>,
                         56.67}
element from tuple: one
ok
3> *** Terminating erlang (nonode@nohost)


<a id="section2b"></a>
### Lists

In [10]:
%%writefile ertest.erl
-module(ertest).
-export([main/0]).

main() ->
    A = [1,2,three],
    io:format("List with mixed content~p~n", [A]),
    % function to get element from list is different to getting from a tuple :(
    io:format("element from list: ~p~n", [lists:nth(3,A)]).


Overwriting ertest.erl


In [11]:
%%script erl
c(ertest).
ertest:main().

Eshell V10.2.3  (abort with ^G)
1> {ok,ertest}
2> List with mixed content[1,2,three]
element from list: three
ok
3> *** Terminating erlang (nonode@nohost)


In [12]:
%%writefile ertest.erl
-module(ertest).
-export([main/0]).

main() ->
    [_,B,_] = [1,2,three],
    io:format("element from list: ~p~n", [B]),
    [A|_] = [1,2,three], 
    io:format("element from list 2: ~p~n", [A]),
    if  B==2 -> 
            io:format("B is equal to 2~n");
        % 2nd condition will only be checked if first one fails
        A==1 ->
            io:format("A is equal to 1~n")
    end.
    

Overwriting ertest.erl


In [13]:
%%script erl
c(ertest).
ertest:main().

Eshell V10.2.3  (abort with ^G)
1> ertest.erl:12: Warning: this clause cannot match because a previous clause at line 9 always matches
{ok,ertest}
2> element from list: 2
element from list 2: 1
B is equal to 2
ok
3> *** Terminating erlang (nonode@nohost)


In [84]:
%%script erl

% first way to append lists
lists:append([1,2,3], [4,5,6]).
    
% second way to append lists
[1,2,3] ++ [4,5,6].

lists:concat([doc, '/', file, '.', 3]).
    
lists:duplicate(500, xx).
    
Fn = fun (X,Y) -> X+Y end.
lists:foldl(Fn,0,lists:seq(1,10)).

% reverse a list using fold1
Fn2 = fun (X,Y) -> [X]++Y end.
lists:foldl(Fn2,[],lists:seq(1,10)).
    


Eshell V10.2.3  (abort with ^G)
1> [1,2,3,4,5,6]
2> [1,2,3,4,5,6]
3> "doc/file.3"
4> [xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,xx,
 xx,xx,xx,xx,xx,xx,xx,xx,xx,xx|...]
5> #Fun<erl_eval.12.128620087>
6> 55
7> #Fun<erl_eval.12.128620087>
8> [10,9,8,7,6,5,4,3,2,1]
9> *** Terminating erlang (nonode@nohost)


Lists module has all the standard functions that you can apply to list
[See here](http://erlang.org/documentation/doc-5.7.4/lib/stdlib-1.16.4/doc/html/lists.html)

In [14]:
%%writefile ertest.erl
-module(ertest).
-export([main/0]).

biggerThan3(N)->
    if 
        N > 3 ->
            true;
        true ->
            false
        % Some people say you should use N =< 3 -> rather than the generic else
    end.

main() ->
    A = [1,2,3,4,5,6,7],
    B = lists:filter(fun biggerThan3/1,A),
    io:format("B = ~p~n",[B]).

Overwriting ertest.erl


In [15]:
%%script erl
c(ertest).
ertest:main().

Eshell V10.2.3  (abort with ^G)
1> {ok,ertest}
2> B = [4,5,6,7]
ok
3> *** Terminating erlang (nonode@nohost)


In [16]:
%%writefile ertest.erl
-module(ertest).
-export([main/0]).

printElement(1)->
    io:format("Element is a 1~n");
printElement(2)->
    io:format("Element is a 2~n");
printElement(_)->
    io:format("Generic catch all for element~n").
main() ->
    A = [1,2,3,4,5,6,7],
    lists:foreach(fun printElement/1,A).

Overwriting ertest.erl


In [17]:
%%script erl
c(ertest).
ertest:main().

Eshell V10.2.3  (abort with ^G)
1> {ok,ertest}
2> Element is a 1
Element is a 2
Generic catch all for element
Generic catch all for element
Generic catch all for element
Generic catch all for element
Generic catch all for element
ok
3> *** Terminating erlang (nonode@nohost)


List comprehension 
[http://erlang.org/doc/programming_examples/list_comprehensions.html](http://erlang.org/doc/programming_examples/list_comprehensions.html)

In [67]:
%%script erl
[X || X <- [1,2,a,3,4,b,5,6], X > 3].

Eshell V10.2.3  (abort with ^G)
1> [a,4,b,5,6]
2> *** Terminating erlang (nonode@nohost)


In [70]:
%%script erl
[X*X || X <- lists:seq(1,10)].

Eshell V10.2.3  (abort with ^G)
1> [1,4,9,16,25,36,49,64,81,100]
2> *** Terminating erlang (nonode@nohost)


<a id="quicksort"></a>
### Quicksort - In 5 lines of code

Not quite the performance of Tony Hoares original but nice to show the expressive power of ErLang

In [125]:
%%writefile ertest.erl
-module(ertest).
-export([main/0]).

time_taken_to_execute(F) -> Start = os:timestamp(),
  R = F(),
  io:format("total time taken ~f milli seconds~n", [timer:now_diff(os:timestamp(), Start) / 1000]),
  R.
        
sort([Pivot|T]) ->
    sort([ X || X <- T, X < Pivot]) ++
    [Pivot] ++
    sort([ X || X <- T, X >= Pivot]);
sort([]) -> [].

main2() ->
    BigRandomList = [rand:uniform()|| _<-lists:seq(1,1000000)],
    sort(BigRandomList).
    
main() ->
    time_taken_to_execute(fun main2/0).

Overwriting ertest.erl


In [126]:
%%script erl
c(ertest).
ertest:main().

Eshell V10.2.3  (abort with ^G)
1> {ok,ertest}
2> total time taken 1955.172000 milli seconds
[1.4552051291749635e-6,5.216707039790158e-6,
 5.9751604871705055e-6,6.403909963759169e-6,
 7.149793547212013e-6,8.125332698782906e-6,
 8.14668451731837e-6,8.875166943811408e-6,
 9.320685278924046e-6,1.1573222177596598e-5,
 1.2392308851194578e-5,1.2970907140807952e-5,
 1.3888649479487647e-5,1.4681952589801206e-5,
 1.5328408696446694e-5,1.532880913390766e-5,
 1.628139354015179e-5,1.718074249923429e-5,
 1.721581818536766e-5,1.7390090642277478e-5,
 1.9241578319784125e-5,2.0399906785484312e-5,
 2.2052369652247883e-5,2.3157170015664086e-5,
 2.3296614640289093e-5,2.4162759161949765e-5,
 2.472934046615638e-5,2.5208838326795657e-5,
 2.5453747143577843e-5|...]
3> *** Terminating erlang (nonode@nohost)


Now let's compare with the standard lists:sort function and see how much slower
the quicksort algorithm above is.

Not as bad as I thought it might be:

sorting 10M items:
* my quicksort took 10M = 26secs, 1M = 2secs
* lists:sort took 10M = 11 secs, 1M = 1 secs

For smaller sized lists the difference are not as great.

I am quite impressed that the recursive quicksort algorithm can sort a list of 10M items!

In [127]:
%%writefile ertest.erl
-module(ertest).
-export([main/0]).

time_taken_to_execute(F) -> Start = os:timestamp(),
  R = F(),
  io:format("total time taken ~f milli seconds~n", [timer:now_diff(os:timestamp(), Start) / 1000]),
  R.
        
main2() ->
    BigRandomList = [rand:uniform()|| _<-lists:seq(1,1000000)],
    lists:sort(BigRandomList).
    
main() ->
    time_taken_to_execute(fun main2/0).

Overwriting ertest.erl


In [128]:
%%script erl
c(ertest).
ertest:main().

Eshell V10.2.3  (abort with ^G)
1> {ok,ertest}
2> total time taken 954.533000 milli seconds
[8.771685058306389e-7,1.2083443651222936e-6,
 1.4760155540027853e-6,1.6597539038709641e-6,
 2.557401025393702e-6,2.7649386423744815e-6,
 3.2898931709990364e-6,5.5379073959915814e-6,
 5.95480401599513e-6,6.360616018552534e-6,
 6.58670163078412e-6,6.731547890104217e-6,
 7.031387517231202e-6,7.468258731990751e-6,
 7.926387496803855e-6,8.83572087040374e-6,
 9.431659162673078e-6,1.1079614647080405e-5,
 1.128304077946396e-5,1.3773212141132696e-5,
 1.4405290130858894e-5,1.5053944432241728e-5,
 1.5336475432170538e-5,1.5427478287199925e-5,
 1.548121500205557e-5,1.6034468631742627e-5,
 1.6475339926436128e-5,1.7872405514762768e-5,
 1.8181366807645816e-5|...]
3> *** Terminating erlang (nonode@nohost)


<a id="timeit"></a>
### timeit module

Now that we have a function for timing functions let's define it in a module so we can use elsewhere

In [8]:
%%writefile timeit.erl
-module(timeit).
-export([time_taken_to_execute/1]).

time_taken_to_execute(F) -> Start = os:timestamp(),
  R = F(),
  io:format("total time taken ~f milli seconds~n", [timer:now_diff(os:timestamp(), Start) / 1000]),
  R.

Writing timeit.erl


In [9]:
%%script erl
c(timeit).

Eshell V10.2.3  (abort with ^G)
1> {ok,timeit}
2> *** Terminating erlang (nonode@nohost)


Test the new timeit module on the sort example again to see that it works

In [133]:
%%writefile ertest.erl
-module(ertest).
-export([main/0]).
 
main2() ->
    BigRandomList = [rand:uniform()|| _<-lists:seq(1,1000000)],
    lists:sort(BigRandomList).
    
main() ->
    timeit:time_taken_to_execute(fun main2/0).

Overwriting ertest.erl


In [135]:
%%script erl
c(ertest).
ertest:main().

Eshell V10.2.3  (abort with ^G)
1> {ok,ertest}
2> total time taken 1016.385000 milli seconds
[1.6427651169381718e-6,2.434061375899965e-6,
 4.36829750238843e-6,5.941004812659578e-6,
 6.437616812404734e-6,7.602887827529159e-6,
 7.99145604690743e-6,9.007899386959117e-6,
 9.123814180056478e-6,9.693376031960632e-6,
 1.1988222241465785e-5,1.2090774061945808e-5,
 1.5182001821267832e-5,1.5521620982839046e-5,
 1.7317840117381245e-5,1.9517357146559e-5,
 2.0105898504096942e-5,2.0917071517856378e-5,
 2.2684729504507395e-5,2.270991822228563e-5,
 2.2741195713438245e-5,2.4246242522285577e-5,
 2.5353065862043778e-5,2.7619397652056676e-5,
 2.7715057547283983e-5,3.0247354138479565e-5,
 3.0663563699961927e-5,3.392934972257233e-5,
 3.4284567098308294e-5|...]
3> *** Terminating erlang (nonode@nohost)


<a id="listTimingTest"></a>
### List timing tests
The results of this timing tests are not what I thought.
It appears that it is faster to add to the end of lists than the front.

In [31]:
%%writefile ertest.erl
-module(ertest).
-export([main/0]).

addToHead(0,L,ToAdd) ->
    L;
addToHead(N,L,ToAdd) ->
    addToHead(N-1,[ToAdd] ++ L,ToAdd).
    
addToTail(0,L,ToAdd) ->
    L;
addToTail(N,L,ToAdd) ->
    addToHead(N-1, L ++ [ToAdd],ToAdd).
    
main1() ->
    addToHead(10000000,[],1).
    
main2() ->
    addToTail(10000000,[],1).
    
main() ->
    io:format("Time to add items to front of list~n"),
    timeit:time_taken_to_execute(fun main1/0),
    io:format("Time to add items to end of list~n"),
    timeit:time_taken_to_execute(fun main2/0).

Overwriting ertest.erl


In [32]:
%%script erl
c(ertest).
ertest:main().

Eshell V10.2.3  (abort with ^G)
1> ertest.erl:4: Warning: variable 'ToAdd' is unused
ertest.erl:9: Warning: variable 'ToAdd' is unused
{ok,ertest}
2> Time to add items to front of list
total time taken 570.257000 milli seconds
Time to add items to end of list
total time taken 225.196000 milli seconds
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1|...]
3> *** Terminating erlang (nonode@nohost)


In [18]:
%%script erl
L = [1,2,3,4].
L ++ [3,4].

Eshell V10.2.3  (abort with ^G)
1> [1,2,3,4]
2> [1,2,3,4,3,4]
3> *** Terminating erlang (nonode@nohost)


<a id="maps"></a>
## Maps

[See Maps Module](http://erlang.org/doc/man/maps.html)

In [18]:
%%writefile ertest.erl
-module(ertest).
-export([main/0]).

main() ->
  M = #{a => 2, b => 3, c=> 4, "a" => 1, "b" => 2, "c" => 4},
  Pred = fun(K,V) -> is_atom(K) andalso (V rem 2) =:= 0 end,
  A = maps:filter(Pred,M),
  io:format("Filtered Map ~p~n",[A]),
  maps:find(b, M).
    

Overwriting ertest.erl


In [19]:
%%script erl
c(ertest).
ertest:main().

Eshell V10.2.3  (abort with ^G)
1> {ok,ertest}
2> Filtered Map #{a => 2,c => 4}
{ok,3}
3> *** Terminating erlang (nonode@nohost)


<a id="section3"></a>
## Functions - with Pattern Matching

In [59]:
%%writefile ertest.erl
-module(ertest).
-export([main/0]).

testFunc(1) ->
    io:format("Simple pattern matching - matches 1~n");
testFunc(2) ->
    io:format("Simple pattern matching - matches 2~n");
testFunc(N) when N == 3 andalso is_integer(N) ->        
    io:format("pattern matching with when clause - matches 3~n");
testFunc(N) when is_integer(N) ->        
    io:format("pattern matching with when clause - Its an integer~n");
testFunc(N) when atom(N) ->        
    io:format("pattern matching with when clause - Its an atom~n").
testFunc(_) -> 
    io:format("pattern matching with when clause - Its an atom~n").
        
main() ->
    testFunc(1),
    testFunc(2),
    testFunc(3),
     testFunc(hello),
    testFunc(4).

Overwriting ertest.erl


In [136]:
%%script erl
c(ertest).
ertest:main().

Eshell V10.2.3  (abort with ^G)
1> {ok,ertest}
2> total time taken 1017.057000 milli seconds
[3.15082269741751e-6,3.514132890347632e-6,
 4.288636711535787e-6,4.7640517919500525e-6,
 5.049995148609376e-6,5.343883956565598e-6,
 5.377471908873943e-6,6.2344732694175065e-6,
 7.388309771760859e-6,9.289194105699394e-6,
 9.70851031778519e-6,1.108534271943018e-5,
 1.2694092762610687e-5,1.4398812239191372e-5,
 1.664376859811245e-5,1.7770082596957337e-5,
 2.1780988871622498e-5,2.182626201330251e-5,
 2.2634497902296324e-5,2.6382063472496498e-5,
 2.6546163937490874e-5,2.6550236417954842e-5,
 2.912367557628137e-5,3.0220456345930735e-5,
 3.071319440595133e-5,3.140945080881696e-5,
 3.310263390321477e-5,3.3326671692535115e-5,
 3.41300098879449e-5|...]
3> *** Terminating erlang (nonode@nohost)


Here are predicate functions you can use in guards to check types:

is_atom/1           is_binary/1        
is_bitstring/1      is_boolean/1        is_builtin/3       
is_float/1          is_function/1       is_function/2      
is_integer/1        is_list/1           is_number/1        
is_pid/1            is_port/1           is_record/2        
is_record/3         is_reference/1      is_tuple/1        

<a id="funcClosure"></a>
### Does Erlang support functional closures?

The answer is yes Erlang does support closures. The next test will prove this.

In [147]:
%%writefile ertest.erl
-module(ertest).
-export([main/0]).

closureTest() ->
    ClosureVar = ["hello", "world",1,2,3],
    fun () -> 
        io:format("ClosureVar = ~p~n", [ClosureVar]) 
    end.
    
main() ->
    Fn = closureTest(),
    Fn().

Overwriting ertest.erl


In [148]:
%%script erl
c(ertest).
ertest:main().

Eshell V10.2.3  (abort with ^G)
1> {ok,ertest}
2> ClosureVar = ["hello","world",1,2,3]
ok
3> *** Terminating erlang (nonode@nohost)


<a id="partialApplication"></a>
### Partial function application in Erlang

A feature of some functional languages like Haskell is to support partial application of functions.
This means that you can call a function with a subset of the parameters that is requires.
The function is only executed when all the parameters have been provided.

Partial application is not supported natively in Erlang but we should be able to simulate it fairly easily

In [42]:
%%writefile ertest.erl
-module(ertest).
-export([main/0]).

eval({Fn,N}) ->
    % Create the initial EvalContext
    {{paramCount,0}, {paramsRequired,N}, {funcCall,Fn}, {params,[]}}.
    
eval(EvalContext,PList) ->
    {{paramCount,PC}, {paramsRequired,PR}, {funcCall,Fn}, {params,ParamList}} = EvalContext,
    NewParamCount = length(PList) + PC,
    NewParamsList = ParamList ++ PList,
    if NewParamCount == PR ->
        Fn(NewParamsList);
        true ->
        io:format("Adding to parameters received ~n"),
        {{paramCount,NewParamCount}, {paramsRequired,PR}, {funcCall,Fn}, {params,NewParamsList}}
    end.

lazyEvalTest([P1,P2,P3]) ->
    io:format("lazyEvalTest = ~p ~p ~p~n", [P1, P2, P3]).
    
main() ->
    Fn = fun lazyEvalTest/1,
    eval(eval(eval(eval({Fn,3}),[1]),[2]),[3]).

Overwriting ertest.erl


In [43]:
%%script erl
c(ertest).
ertest:main().

Eshell V10.2.3  (abort with ^G)
1> ertest.erl:23: Warning: variable 'Fn' is unused
{ok,ertest}
2> Adding to parameters received 
Adding to parameters received 
lazyEvalTest = 1 2 3
ok
3> *** Terminating erlang (nonode@nohost)


In [1]:
%%script erl
L1 = [1,2,3,4,5].
length(L1).

Eshell V10.2.3  (abort with ^G)
1> [1,2,3,4,5]
2> 5
3> *** Terminating erlang (nonode@nohost)


In [33]:
%%writefile ertest.erl
-module(ertest).
-export([main/0,lazyEvalTest/3]).

evalInit(Fn,N) ->
    % Create the initial EvalContext
    {{paramCount,0}, {paramsRequired,N}, {funcCall,Fn}, {params,[]}}.
    
eval(EvalContext,PList) ->
    {{paramCount,PC}, {paramsRequired,PR}, {funcCall,Fn}, {params,ParamList}} = EvalContext,
    NewParamCount = length(PList) + PC,
    NewParamsList = ParamList ++ PList,
    if NewParamCount == PR ->
        % Fn(NewParamsList);
        apply(?MODULE, lazyEvalTest, NewParamsList);
        true ->
        io:format("Adding to parameters received ~n"),
        {{paramCount,NewParamCount}, {paramsRequired,PR}, {funcCall,Fn}, {params,NewParamsList}}
    end.

lazyEvalTest(P1,P2,P3) ->
    io:format("lazyEvalTest = ~p ~p ~p~n", [P1, P2, P3]).
    
main() ->
    Fn = fun lazyEvalTest/3,
    C = evalInit(Fn,3),
    eval(eval(eval(C,[1]),[2]),[3]).

Overwriting ertest.erl


In [35]:
%%script erl
c(ertest).
ertest:main().

Eshell V10.2.3  (abort with ^G)
1> {ok,ertest}
2> Adding to parameters received 
Adding to parameters received 
lazyEvalTest = 1 2 3
ok
3> *** Terminating erlang (nonode@nohost)


In [39]:
%%writefile ertest.erl
-module(ertest).
-export([main/0]).

testIt()->
    self() ! hello,
    self() ! hello,
    self() ! boom,
    c:flush().
        
main() ->
    testIt().

Overwriting ertest.erl


In [40]:
%%script erl
c(ertest).
ertest:main().

Eshell V10.2.3  (abort with ^G)
1> {ok,ertest}
2> Shell got hello
Shell got hello
Shell got boom
ok
3> *** Terminating erlang (nonode@nohost)


<a id="debugger"></a>
## Debugging an Erlang program


In [63]:
%%writefile ertest.erl
-module(ertest).
-export([main/0]).

testIt()->
    io:format("Line 1~n"),
    io:format("Line 2~n"),
    io:format("Line 3~n"),
    io:format("Line 4~n"),
    io:format("Line 5~n").         
        
main() ->
    testIt().

Overwriting ertest.erl


In [64]:
%%script erl
c(ertest,[debug_info]).
debugger:start().
ertest:main().

Eshell V10.2.3  (abort with ^G)
1> {ok,ertest}
2> {ok,<0.83.0>}
3> Line 1
Line 2
Line 3
Line 4
Line 5
ok
4> *** Terminating erlang (nonode@nohost)


<a id="concurrent"></a>
## Concurrent Programming

See [http://erlang.org/doc/getting_started/conc_prog.html](http://erlang.org/doc/getting_started/conc_prog.html)

In [71]:
%%writefile ertest.erl
-module(ertest).

-export([main/0, say_something/2]).

say_something(What, 0) ->
    done;
say_something(What, Times) ->
    io:format("~p~n", [What]),
    say_something(What, Times - 1).

main() ->
    spawn(ertest, say_something, [hello, 3]),
    spawn(ertest, say_something, [goodbye, 3]).

Overwriting ertest.erl


In [72]:
%%script erl
c(ertest).
ertest:main().

Eshell V10.2.3  (abort with ^G)
1> ertest.erl:5: Warning: variable 'What' is unused
{ok,ertest}
2> hello
goodbye
hello
goodbye
<0.84.0>
hello
goodbye
3> *** Terminating erlang (nonode@nohost)


### Ping Pong - Message passing example

In [135]:
%%writefile tut15.erl
-module(tut15).

-export([start/0, ping/2, pong/0, processStatus/1, waitTillFinished/1]).

ping(0, Pong_PID) ->
    Pong_PID ! finished,
    io:format("ping finished~n", []);

ping(N, Pong_PID) ->
    Pong_PID ! {ping, self()},
    receive
        pong ->
            io:format("Ping received pong ~p~n", [N])
    end,
    ping(N - 1, Pong_PID).

pong() ->
    receive
        finished ->
            io:format("Pong finished~n", []);
        {ping, Ping_PID} ->
            io:format("Pong received ping~n", []),
            Ping_PID ! pong,
            pong()
    end.

processStatus(Pid)->
    PStatus = process_info(Pid),
    if is_list(PStatus) ->
        St = lists:keyfind(status, 1, PStatus),
        element(2, St);  
    true ->
        finished
    end.

waitTillFinished(Pid) ->
    Status = processStatus(Pid),
    case Status of
        St when St =:= running; St =:= waiting -> 
            io:format("Wait for Process ~p to finish ~p~n",[Pid,St]),
            timer:sleep(1),
            waitTillFinished(Pid);
        finished ->
            finished
    end.
    
start() ->
    Pong_PID = spawn(tut15, pong, []),
    spawn(tut15, ping, [30, Pong_PID]).

Overwriting tut15.erl


In [134]:
%%script erl
c(tut15).
Pid = tut15:start().
tut15:waitTillFinished(Pid).   

Eshell V10.2.3  (abort with ^G)
1> {ok,tut15}
2> Pong received ping
<0.84.0>
Ping received pong 3000
3> Pong received ping
Ping received pong 2999
Wait for Process <0.84.0> to finish running
Pong received ping
Ping received pong 2998
Pong received ping
Ping received pong 2997
Pong received ping
Ping received pong 2996
Pong received ping
Ping received pong 2995
Pong received ping
Ping received pong 2994
Pong received ping
Ping received pong 2993
Pong received ping
Ping received pong 2992
Pong received ping
Ping received pong 2991
Pong received ping
Ping received pong 2990
Pong received ping
Ping received pong 2989
Pong received ping
Ping received pong 2988
Pong received ping
Ping received pong 2987
Pong received ping
Ping received pong 2986
Pong received ping
Ping received pong 2985
Pong received ping
Ping received pong 2984
Pong received ping
Ping received pong 2983
Pong received ping
Ping received pong 2982
Pong received ping
Ping received pong 2981
Pong received ping
Ping received po